In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
asoiaf = pickle.load( open( "./Data/further_cleaned_corpus_asoiaf.pkl", "rb" ) )

In [32]:
final_asoiaf = []

for item in asoiaf:
    item = ' '.join(item)
    final_asoiaf.append(item)

### Count Vectorizer ###

In [33]:
vectorizer = CountVectorizer(stop_words='english')
doc_word = vectorizer.fit_transform(final_asoiaf)
doc_word.shape

(24743, 5164)

### LSA/PCA/SVD ###

In [39]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.1687657 , 0.03539193, 0.03051448, 0.02645989, 0.02193389])

In [40]:
topic_word = pd.DataFrame(lsa.components_.round(5),
             index = ["component_1","component_2", 'component_3', 'component_4', 'component_5'],
             columns = vectorizer.get_feature_names())
topic_word

,abandon,abandoned,abandoning,ability,able,abomination,absence,absent,absolute,absolutely,...,young,younger,youngest,youth,youtu,youtube,yunkai,zero,zeus,zombie
component_1,0.00272,0.00164,0.00082,0.01084,0.02365,0.00098,0.00113,0.00118,0.00269,0.00793,...,0.01351,0.00429,0.00082,0.00183,0.00560,0.01015,0.00308,0.00272,0.00045,0.00600
component_2,0.00139,0.00009,0.00039,-0.00538,-0.00892,-0.00089,-0.00014,0.00105,0.00058,0.00092,...,0.01833,0.01403,0.00151,0.00323,0.00167,0.00041,0.00519,-0.00077,-0.00007,-0.00183
component_3,-0.00412,-0.00048,-0.00041,0.01603,0.00540,0.00219,-0.00096,0.00089,-0.00220,-0.00461,...,0.01106,0.00934,0.00034,0.00065,-0.00314,-0.00493,-0.00769,-0.00197,0.00044,-0.00285
component_4,0.00004,0.00166,0.00070,0.01642,0.01377,0.00179,-0.00189,-0.00303,0.00119,0.00566,...,0.01976,0.00104,0.00103,0.00449,0.00326,0.01163,0.00858,0.00026,-0.00034,0.00170
component_5,0.00186,0.00013,-0.00032,-0.00390,0.00239,0.00105,0.00096,-0.00029,-0.00010,0.00277,...,0.00832,0.00158,0.00112,0.00501,0.00969,0.01003,0.01741,-0.00048,0.00205,-0.00064


In [41]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [42]:
display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
king, jon, night, bran, dany, dragon, time, arya, cersei, think

Topic  1
jaime, hand, cersei, tyrion, dany, daenerys, lannister, ser, think, sansa

Topic  2
bran, hand, jaime, king, crow, eyed, night, time, dream, euron

Topic  3
bran, time, crow, euron, dream, eyed, know, dany, think, story

Topic  4
king, dany, night, dragon, daenerys, city, westeros, child, house, men


In [43]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = final_asoiaf,
             columns = ["component_1","component_2", 'component_3', 'component_4', 'component_5'])


In [44]:
ex_label = [e[:30]+"..." for e in final_asoiaf]
vectorizer = CountVectorizer(stop_words = 'english')
doc_word = vectorizer.fit_transform(final_asoiaf)
pd.DataFrame(doc_word.toarray(), index=ex_label, columns=vectorizer.get_feature_names()).head(10)

,abandon,abandoned,abandoning,ability,able,abomination,absence,absent,absolute,absolutely,...,young,younger,youngest,youth,youtu,youtube,yunkai,zero,zeus,zombie
main castle black importance h...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
main craster king carry king b...,0,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
main living able defeat enemy ...,0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
main game thrones’ final pull ...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
inconsistency castle black sig...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
da bran e rhaegar e jon snow b...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
much time make prediction thin...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
upon adaptation main rewatchin...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
random post...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
grrm expressed desire make pro...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### NMF ###

In [45]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(doc_word)

In [46]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2", 'component_3', 'component_4', 'component_5'],
             columns = vectorizer.get_feature_names())
list(topic_word.columns)

['abandon',
 'abandoned',
 'abandoning',
 'ability',
 'able',
 'abomination',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abused',
 'abusive',
 'ac',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accident',
 'accidentally',
 'accompanied',
 'accomplish',
 'accomplished',
 'accomplishment',
 'according',
 'account',
 'accuracy',
 'accurate',
 'accused',
 'achieve',
 'achieved',
 'achievement',
 'achieving',
 'acknowledge',
 'acok',
 'act',
 'acted',
 'acting',
 'action',
 'active',
 'actively',
 'activity',
 'actor',
 'actress',
 'actual',
 'actually',
 'ad',
 'adapt',
 'adaptation',
 'adapted',
 'adapting',
 'add',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'addressed',
 'admit',
 'admits',
 'admitted',
 'admittedly',
 'ado',
 'adopted',
 'adult',
 'advance',
 'advanced',
 'advantage',
 'adventure',
 'advice',
 'advise',
 'advised',
 'adviser',
 'advisor',
 'aegon

In [47]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
king, night, walker, white, army, dead, battle, bran, dragon, winterfell

Topic  1
jaime, hand, cersei, tyrion, king, lannister, brienne, ser, affc, asos

Topic  2
bran, crow, time, eyed, euron, dream, know, weirwood, god, bloodraven

Topic  3
dany, dragon, daenerys, people, think, time, know, way, make, throne

Topic  4
jon, arya, sansa, battle, winterfell, think, make, cersei, kill, snow


In [48]:
H = pd.DataFrame(doc_topic.round(5),
             index = ex_label,
             columns = ["component_1","component_2", 'component_3', 'component_4', 'component_5'])

In [49]:
H

,component_1,component_2,component_3,component_4,component_5
main castle black importance h...,0.11627,0.04615,0.03909,0.05900,0.03662
main craster king carry king b...,1.57329,0.36241,0.00000,0.58576,0.00000
main living able defeat enemy ...,0.22891,0.00000,0.00000,0.14757,0.00000
main game thrones’ final pull ...,0.00038,0.00000,0.00296,0.01332,0.00341
inconsistency castle black sig...,0.11572,0.04682,0.03913,0.05781,0.03482
da bran e rhaegar e jon snow b...,0.00000,0.00000,0.05648,0.00000,0.12015
much time make prediction thin...,0.03514,0.05924,0.02943,0.16462,0.13860
upon adaptation main rewatchin...,0.00000,0.01504,0.01147,0.04268,0.09503
random post...,0.00000,0.00042,0.00027,0.00104,0.00033
grrm expressed desire make pro...,0.00000,0.00162,0.00217,0.01017,0.00302


### LDA ###

In [50]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer.fit(final_asoiaf)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [51]:
doc_word = count_vectorizer.transform(final_asoiaf).transpose()

In [52]:
pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).sample(10)

,0,1,2,3,4,5,6,7,8,9,...,24733,24734,24735,24736,24737,24738,24739,24740,24741,24742
choice little,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
learn guy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
prophecy bastard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
power skill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
believable fallout,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dragonstone nuclear,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
complex fantasy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
final sit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
isn joke,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
great assuming,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
doc_word.shape

(503565, 24743)

In [54]:
from gensim import corpora, models, similarities, matutils

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [55]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

In [56]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [57]:
len(id2word)

503565

In [59]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=1) # train model

2019-05-19 11:55:48,097 : INFO : using symmetric alpha at 0.2
2019-05-19 11:55:48,098 : INFO : using symmetric eta at 0.2
2019-05-19 11:55:48,196 : INFO : using serial LDA version on this node
2019-05-19 11:55:48,427 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 24743 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-05-19 11:55:48,478 : INFO : PROGRESS: pass 0, at document #2000/24743
2019-05-19 11:55:50,008 : INFO : merging changes from 2000 documents into a model of 24743 documents
2019-05-19 11:55:50,232 : INFO : topic #0 (0.200): 0.007*"king" + 0.004*"jon" + 0.003*"think" + 0.003*"time" + 0.003*"throne" + 0.003*"dany" + 0.002*"game" + 0.002*"night" + 0.002*"jaime" + 0.002*"main"
2019-05-19 11:55:50,235 : INFO : topic #1 (0.200): 0.004*"jon" + 0.004*"king" + 0.004*"night" + 0.003*"dragon" + 0.003*"think" + 0.003*"dany" + 0.

2019-05-19 11:55:59,475 : INFO : merging changes from 2000 documents into a model of 24743 documents
2019-05-19 11:55:59,685 : INFO : topic #0 (0.200): 0.007*"main" + 0.006*"king" + 0.005*"throne" + 0.004*"night" + 0.004*"think" + 0.004*"game" + 0.004*"people" + 0.004*"jon" + 0.003*"got" + 0.003*"thing"
2019-05-19 11:55:59,688 : INFO : topic #1 (0.200): 0.011*"night" + 0.009*"king" + 0.007*"night king" + 0.007*"jon" + 0.005*"walker" + 0.005*"white" + 0.004*"white walker" + 0.004*"dragon" + 0.004*"bran" + 0.003*"kill"
2019-05-19 11:55:59,695 : INFO : topic #2 (0.200): 0.005*"cersei" + 0.005*"arya" + 0.005*"jon" + 0.004*"tyrion" + 0.004*"dany" + 0.003*"sansa" + 0.003*"kill" + 0.003*"jaime" + 0.002*"make" + 0.002*"know"
2019-05-19 11:55:59,698 : INFO : topic #3 (0.200): 0.004*"think" + 0.004*"story" + 0.004*"time" + 0.003*"dany" + 0.003*"know" + 0.003*"dragon" + 0.003*"grrm" + 0.003*"jon" + 0.002*"people" + 0.002*"end"
2019-05-19 11:55:59,701 : INFO : topic #4 (0.200): 0.008*"bran" + 0.00

2019-05-19 11:56:08,651 : INFO : topic #1 (0.200): 0.009*"night" + 0.007*"king" + 0.006*"night king" + 0.005*"jon" + 0.004*"walker" + 0.004*"white" + 0.004*"white walker" + 0.003*"kill" + 0.003*"dragon" + 0.003*"arya"
2019-05-19 11:56:08,659 : INFO : topic #2 (0.200): 0.007*"cersei" + 0.005*"tyrion" + 0.004*"dany" + 0.004*"arya" + 0.004*"jaime" + 0.003*"jon" + 0.003*"kill" + 0.002*"people" + 0.002*"sansa" + 0.002*"make"
2019-05-19 11:56:08,662 : INFO : topic #3 (0.200): 0.004*"think" + 0.003*"time" + 0.003*"grrm" + 0.003*"story" + 0.003*"dany" + 0.002*"know" + 0.002*"dragon" + 0.002*"euron" + 0.002*"really" + 0.002*"people"
2019-05-19 11:56:08,665 : INFO : topic #4 (0.200): 0.007*"jon" + 0.006*"bran" + 0.004*"dany" + 0.003*"dragon" + 0.003*"winterfell" + 0.003*"king" + 0.003*"battle" + 0.003*"army" + 0.003*"dead" + 0.002*"wight"
2019-05-19 11:56:08,674 : INFO : topic diff=0.245753, rho=0.277350


In [60]:
lda.print_topics()

2019-05-19 11:56:49,657 : INFO : topic #0 (0.200): 0.005*"really" + 0.004*"people" + 0.004*"think" + 0.004*"good" + 0.004*"throne" + 0.004*"thing" + 0.003*"make" + 0.003*"got" + 0.003*"way" + 0.003*"king"
2019-05-19 11:56:49,661 : INFO : topic #1 (0.200): 0.009*"night" + 0.007*"king" + 0.006*"night king" + 0.005*"jon" + 0.004*"walker" + 0.004*"white" + 0.004*"white walker" + 0.003*"kill" + 0.003*"dragon" + 0.003*"arya"
2019-05-19 11:56:49,668 : INFO : topic #2 (0.200): 0.007*"cersei" + 0.005*"tyrion" + 0.004*"dany" + 0.004*"arya" + 0.004*"jaime" + 0.003*"jon" + 0.003*"kill" + 0.002*"people" + 0.002*"sansa" + 0.002*"make"
2019-05-19 11:56:49,671 : INFO : topic #3 (0.200): 0.004*"think" + 0.003*"time" + 0.003*"grrm" + 0.003*"story" + 0.003*"dany" + 0.002*"know" + 0.002*"dragon" + 0.002*"euron" + 0.002*"really" + 0.002*"people"
2019-05-19 11:56:49,675 : INFO : topic #4 (0.200): 0.007*"jon" + 0.006*"bran" + 0.004*"dany" + 0.003*"dragon" + 0.003*"winterfell" + 0.003*"king" + 0.003*"battle" 

[(0,
  '0.005*"really" + 0.004*"people" + 0.004*"think" + 0.004*"good" + 0.004*"throne" + 0.004*"thing" + 0.003*"make" + 0.003*"got" + 0.003*"way" + 0.003*"king"'),
 (1,
  '0.009*"night" + 0.007*"king" + 0.006*"night king" + 0.005*"jon" + 0.004*"walker" + 0.004*"white" + 0.004*"white walker" + 0.003*"kill" + 0.003*"dragon" + 0.003*"arya"'),
 (2,
  '0.007*"cersei" + 0.005*"tyrion" + 0.004*"dany" + 0.004*"arya" + 0.004*"jaime" + 0.003*"jon" + 0.003*"kill" + 0.002*"people" + 0.002*"sansa" + 0.002*"make"'),
 (3,
  '0.004*"think" + 0.003*"time" + 0.003*"grrm" + 0.003*"story" + 0.003*"dany" + 0.002*"know" + 0.002*"dragon" + 0.002*"euron" + 0.002*"really" + 0.002*"people"'),
 (4,
  '0.007*"jon" + 0.006*"bran" + 0.004*"dany" + 0.003*"dragon" + 0.003*"winterfell" + 0.003*"king" + 0.003*"battle" + 0.003*"army" + 0.003*"dead" + 0.002*"wight"')]

In [61]:
from gensim.test.utils import datapath

# Save model to disk.
lda_asoiaf = datapath("model")
lda.save(lda_asoiaf)

# Load a potentially pretrained model from disk.
#lda = LdaModel.load(temp_file)

2019-05-19 11:58:53,189 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-05-19 11:58:53,190 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2019-05-19 11:58:53,191 : INFO : saving LdaState object under /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model.state, separately None
2019-05-19 11:58:53,193 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-19 11:58:53,299 : INFO : saved /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model.state
2019-05-19 11:58:53,301 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-19 11:58:55,647 : INFO : saving LdaModel object under /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model, separately ['expElogbeta', 'sstats']
2019-05-19 11:58:55,648 : INFO : storing np array 'expElogbeta' to /anaconda3/lib/python3.7/site-packages/gensim/test/test

In [63]:
lda = models.LdaModel.load(lda_asoiaf)

2019-05-19 12:02:39,718 : INFO : loading LdaModel object from /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model
2019-05-19 12:02:39,720 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-19 12:02:39,731 : INFO : loading expElogbeta from /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model.expElogbeta.npy with mmap=None
2019-05-19 12:02:39,740 : INFO : setting ignored attribute dispatcher to None
2019-05-19 12:02:39,741 : INFO : setting ignored attribute state to None
2019-05-19 12:02:39,742 : INFO : setting ignored attribute id2word to None
2019-05-19 12:02:39,743 : INFO : loaded /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model
2019-05-19 12:02:39,745 : INFO : loading LdaState object from /anaconda3/lib/python3.7/site-packages/gensim/test/test_data/model.state
2019-05-19 12:02:39,746 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-19 12:02:39,875 : INFO : loaded /anaconda3/lib/python

In [25]:
#lda = LdaModel(corpus, num_topics=100)  # train model
#print(lda[doc_bow]) # get topic probability distribution for a document
#lda.update(corpus2) # update the LDA model with additional documents
#print(lda[doc_bow])

#lda = LdaModel(corpus, num_topics=50, alpha='auto', eval_every=5)  # train asymmetric alpha from data

### IF IDF Vectorizer ###